# Covered here

* [Scopes & namespaces](#Scopes-&-namespaces)
* [Object orientation](#Object-orientation)
* [Intro to classes](#Intro-to-classes)
* [More on method objects](#More-on-method-objects)
* [Share data](#Share-data)
* [Intermediate class conventions](#Intermediate-class-conventions)
* [Class inheritance](#Class-inheritance)
* [Private variables](#Private-variables)

# Resources & references

* Docs: [Python Scopes & Namespaces](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)
* [Google Style Guide: Lexical Scoping](https://google.github.io/styleguide/pyguide.html#Lexical_Scoping)
* [PEP 227 -- Statically Nested Scopes](https://www.python.org/dev/peps/pep-0227/)
* [PEP 3104 -- Access to Names in Outer Scopes](https://www.python.org/dev/peps/pep-3104/)
* farblog: [Understanding Python Scope](https://www.farside.org.uk/201307/understanding_python_scope)
* [Gotcha: Python, scoping, and closures](https://eev.ee/blog/2011/04/24/gotcha-python-scoping-closures/)
* Stack Overflow: 
 * [Lexical Closures in Python](https://stackoverflow.com/questions/233673/lexical-closures-in-python)
 * [Calling a function within itself](https://stackoverflow.com/questions/29073110/python-calling-a-function-from-within-itself)
 * [Bascis of Recursion in Python](https://stackoverflow.com/questions/30214531/basics-of-recursion-in-python)
 
On **`Super()`**:
- The builtin [`super()`](https://docs.python.org/3/library/functions.html#super)
- [What's new in Python 2.2: Multiple Inheritance--The Diamond Rule](https://docs.python.org/3/whatsnew/2.2.html#multiple-inheritance-the-diamond-rule)
- [PEP 3135](https://www.python.org/dev/peps/pep-3135/), detailing the switch in `super()` syntax from Python 2 to 3.
- PEP 253:
    - [Multiple inheritance](https://www.python.org/dev/peps/pep-0253/#multiple-inheritance)
    - [Method resolution order (MRO)](https://www.python.org/dev/peps/pep-0253/#mro-method-resolution-order-the-lookup-rule)
- Raymond Hettinger: [Python’s super() considered super!](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/)
- [Dependency injection](https://stackoverflow.com/a/33469090/7954504) (warning: this gets a bit hairy)
- [Calling `super()` in parent class](https://stackoverflow.com/q/47660676/7954504)

# Scopes & namespaces

**Namespace**: a mapping from **names to objects**. 

Most namespaces are currently implemented as Python dictionaries.  The important thing to know about namespaces is that there is absolutely no relation between names in different namespaces; for instance, two different modules may both define a function `maximize` without confusion — users of the modules just must prefix it with the module name.

Viewing built-in names:

In [4]:
print(dir(__builtins__)[:100])  # ...

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError

**The local namespace for a function is created when the function is called**, and deleted when the function returns or raises an exception that is not handled within the function. (Actually, forgetting would be a better way to describe what actually happens.) Of course, recursive invocations each have their own local namespace.

At any time during execution, there are at least 3 nested scopes whose namespaces are directly accessible:

* the innermost scope, which is searched first, contains the local names
* the scopes of any enclosing functions, which are searched starting with the nearest enclosing scope, contains non-local, but also non-global names
* the next-to-last scope contains the current module’s global names
* the outermost scope (searched last) is the namespace containing built-in names

## Global v. local

Variables declared outside the function can be referenced within the function:

In [1]:
x = 5
def addx(y):
    return x + y
addx(10)

15

But these “global” variables cannot bemodified within the function, unless declared global in the function.
This doesn’t work:

In [8]:
def setx(y):
    x = y
    print('x is %d ' % x)
setx(10)

x is 10 


In [4]:
x

5

This works:

In [6]:
def setx(y):
    global x
    x = y
    print('x is %d ' % x)
setx(10)

x is 10 


In [7]:
x

10

One more example:

In [1]:
a = 5
def f():
    global a
    a = 6
    print(a)
f()
print(a)

6
6


Consider this with `global` excluded:

In [2]:
a = 5
def f():
    a = 6
    print(a)
f()
print(a)

6
5


## What does it mean that Python is _statically scoped_?

This simply means that every mention of a variable name in a program can be resolved ahead of time in order to determine the object to which it refers, by inspection _only_ of the program’s text.

Contrast with _dynamic scope_, where the identity of the object to which a variable refers cannot be determined until runtime.

So, Python is both statically scoped but also lacks an explicit variable declaration statement (e.g. a `var`-like keyword). This means that it needs to know how to determine, when given a statement that refers to a variable, and the existence of a non-local binding with the same name, whether or not that statement creates a new local binding that shadows the existing one.

The rule that Python chooses is: any assignment within a block establishes a new local binding, unless a global statement for the name appears in the block, in which case the name always refers to a binding in the module-global environment instead.

In [4]:
def test_function_calls_callback():
    callback_called = False
    def callback():
        # This isn't re-assigning to `callback_called`.  
        # It's creating a new local binding
        callback_called = True
        print(callback_called)
    callback()
    print(callback_called)
test_function_calls_callback()

True
False


To override this feature you would use `nonlocal`:

In [1]:
def test_function_calls_callback():
    callback_called = False
    def callback():
        nonlocal callback_called
        callback_called = True
        print(callback_called)
    callback()
    print(callback_called)
test_function_calls_callback()

True
True


## Lexical scoping

**Lexical scoping**: variable bindings are resolved based on the static program text.

A nested Python function can refer to variables defined in enclosing functions, but can not assign to them.  Any assignment to a name in a block will cause Python to treat all references to that name as a local variable, even if the use precedes the assignment. If a global declaration occurs, the name is treated as a global variable.

So, **static scope is effectively synonymous with lexical scope**, though in some cases the latter is used to differentiate the subset of statically-scoped languages that allow arbitrary nested scopes, where name resolution is permitted to access bindings defined in a (typically closest) parent scope.

The Python 2.0 definition specified exactly three namespaces to check for each name:
1. local namespace
2. global namespace
3. builtin namespace.  

According to this definition, if a function A is defined within a function B, the names bound in B are not visible in A.  

PEP 227 changes the rules so that names bound in B are visible in A (unless A contains a name binding that hides the binding in B).

### Reference versus assignment

From the Google Style Guide on lexical scoping:

> A nested Python function can refer to variables defined in enclosing functions, but can not assign to them.

In [7]:
# Reference
# ---------
def toplevel():
    a = 5
    def nested():
        print(a + 2)  # nested func can refer to variables def
                      #     in enclosing functions
    nested()
    return a
print(toplevel())

7
5


In [8]:
# Assignment
# ---------
def toplevel():
    a = 5
    def nested():
        a = 7        # "outer" a is still 5, can't modify 
                     #     enclosing scope variable
    nested()
    return a
print(toplevel())

5


So, why does a _combination of reference & assignment_ fail?

In [9]:
# Reference and assignment
def toplevel():
    a = 5
    def nested():
        print(a + 2)
        a = 7
    nested()
    return a
# toplevel()
# UnboundLocalError: local variable 'a' referenced before assignment

In the **reference** case above, Python sees that there is no local `a` and so `a + 2` refers to `toplevel`'s `a`.

In the **assignment** case above, two different `a` variables are assigned.

But in the third case, the whole function `nested()` is checked before it runs.  Therefore `a+2` "knows" that a variable `a` is defined later, but that it hasn't been defined _yet_.  You could fix this with `nonlocal`:

In [11]:
def toplevel():
    a = 5
    def nested():
        nonlocal a
        print(a + 2)
        a = 7        # We're now assigning to the nonlocal a
    nested()
    return a
print(toplevel())

7
7


### Pitfall of lexical scoping

But, one pitfall of lexical scoping is demonstrated below.  The call to `bar()` will refer to the variable `i` bound in `foo()` by the `for` loop.  If `bar()` is called before the loop is executed, a `NameError` will be raised.

In [2]:
i = 4
def foo(x):
    def bar():
        print(i)
    for i in x:  
        # Ah, i *is* now local to `foo`, so this is what `bar` sees
        print(i)
    bar()
    
# foo([1, 2, 3]) will print 1 2 3 3, not 1 2 3 4.
foo([1, 2, 3])

1
2
3
3


# Object orientation

* Python uses an object model; it is object-oriented.  
 * Formally, an object is a collection of data and associated behaviors.
 * Every number, string, data structure, function, class, module, and so on exists in the Python interpreter in its own “box” which is referred to as a Python object.  Each object has an associated type (for example, string or function) and internal data.
* Objects in Python typically have both _attributes_ (other Python objects stored “inside” the objects) and _methods_ (functions associated with an object that have access to the object’s internal content).  A method is a function that “belongs to” an object. (In Python, the term method is not unique to class instances: other object types can have methods as well. For example, list objects have methods called append, insert, remove, sort, and so on.
* A general Python object a (e.g. a variable, function, class instance) be used as a function argument f(a), or can have methods (functions) applied to it, with dot syntax a.f().

## When to Use OOP

A usable definition of an object is that it is a **collection of data and associated behaviors.**  Behaviors are actions that
can occur on an object. The behaviors that can be performed on a specific class of objects are called methods.

Objects are things that have both data and behavior.
* If we are working only with data, we are often better off storing it in a list, set, dictionary, or some other Python data structure). 
* On the other hand, if we are working only with behavior, but no stored data, a simple function is more suitable.

Proficient Python programmers use built-in data structures unless (or until) there is an obvious need to define a class.

As a last word on the issue: _code length is not a good indicator of code complexity_.

# Intro to classes

There’s a construct in Python _called_ a class that lets you structure your software in a particular way.  Classes can be used to add consistency to your programs so that they can be used in a cleaner way.  Classes are used instead of modules because you can take the class and use it to craft many of them, and they won’t interfere with each other.

A basic class consists only of the `class` keyword, the name of the class, and the class from which the new class inherits in parentheses. (ex: `class NewClass(object):`)  By convention, user-defined Python class names start with a capital letter.


In [2]:
# Simplest form of a class
class ClassName:
    #<statement-1>
    #.
    #.
    #.
    #<statement-N>
    pass

In [13]:
# The Student class has __init__, set_age and set_major methods
# Attributes are name, age, and major
class Student(object):
    def __init__(self, name):
        self.name = name
    def set_age(self, age):
        self.age = age
    def set_major(self, major):
        self.major = major

In [14]:
anna = Student('anna')

In [4]:
anna.set_age(21)

In [5]:
anna.set_major('physics')

In [6]:
anna.age

21

In [7]:
anna.set_age # just a method when no parentheses are used

<bound method Student.set_age of <__main__.Student object at 0x000000000AAFCEF0>>

The more normal way to set attributes would be directly:

In [16]:
anna.age = 22
anna.age

22

Class objects support two kinds of operations: attribute references and instantiation.

Attribute references use the standard syntax used for all attribute references in Python: `obj.name`. Valid attribute names are all the names that were in the class’s namespace when the class object was created. So, if the class definition looked like this:

In [3]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

...then `MyClass.i` and `MyClass.f` are valid attribute references, returning an integer and a function object, respectively. Class attributes can also be assigned to, so you can change the value of `MyClass.i` by assignment. `__doc__` is also a valid attribute, returning the docstring belonging to the class: `"A simple example class"`.

In [4]:
MyClass.i

12345

In [6]:
MyClass.f

<function __main__.MyClass.f>

In [7]:
MyClass.__doc__

'A simple example class'

## `__init__()`

For the example above (`MyClass`), the instantiation operation (“calling” a class object) will create an empty object.  Many classes like to create objects with instances customized to a specific initial state. Therefore a class may define a special method named `__init__()`.  When a class defines an `__init__()` method, class instantiation automatically invokes `__init__()` for the newly-created class instance.

* Consider the `__init__()` function required for classes.  It’s used to initialize the objects it creates.  `__init__()` is simply a method of a class; it is the method that “boots up” each object the class creates.
* The `__init__()` function always takes at least one argument, `self`, that refers to the object being created.  **The self argument to a method is simply a reference to the object that the method is being invoked on.**
 * If you don’t have self, then code like won’t be clear as to whether you mean the instance’s class or a local variable.
 * There's nothing magic about the word `self`.  However, it's overwhelmingly common to use `self` as the first parameter in `__init__()`, so you should do this so that other people will understand your code.
* Any method defined within the class should have self as its first argument: `def earn(self, y):`.

See also: 
* [The Self Variable Explained](https://pythontips.com/2013/08/07/the-self-variable-in-python-explained/)

Using `__init__` is simply a way to specify attributes by default, when a class object is created:

In [18]:
# Case 1 - no __init__
class Point:
    def move(self, x, y):
        self.x = x
        self.y = y
    def reset(self):
        self.move(0, 0)

p1 = Point()
p1.reset()

In [17]:
# Case 1 - use __init__
class Point:
    def __init__(self, x=0, y=0):
        self.move(x, y)               # note that you can refer to a "later defined" method within __init__
    def move(self, x, y):
        self.x = x
        self.y = y
    def reset(self):
        self.move(0, 0)

p2 = Point()
p2.x, p2.y

(0, 0)

## Do you need `return self`?

The short answer is that, in most cases, you do not.  Consider the following:

In [3]:
class Point:
    def reset(self):
        self.x = 0
        self.y = 0

p = Point()
p.reset()
p.x, p.y

(0, 0)

In [5]:
class Point:
    def reset(self):
        self.x = 0
        self.y = 0
        return self      # unnecessary here; you are already modifying the object

p2 = Point()
p.reset()
p.x, p.y

(0, 0)

The primary use of `return self` is to encourage method chaining (method cascading).  See [Why use `return self`?](https://stackoverflow.com/questions/43380042/purpose-of-return-self-python).  For example, The `fit` method of all scikit-learn models returns `self`, allowing you to use

rather than

## Difference between classes and objects

Classes describe objects. They are like blueprints for creating an object.  You might have three oranges sitting on the table in front of you. Each orange is a distinct object, but all three have the attributes and behaviors associated with one class: the general class of oranges.

# More on method objects

For various reasons, **functions that are part of classes are referred to as _methods_**.

When you work with classes, **name the methods as if they are commands you're giving to the class**, rather than naming the methods after what the function does:

In [12]:
class newlist(list):
    # Not great...
    def remove_from_end_of_list(self):
        pass
    # Much better...
    def pop(self):
        pass

Recall the example:

In [8]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

Say that you define `x = MyClass()`.  What exactly happens you call `x.f()`, without an argument, even though the function definition for f() specified an argument? Surely Python raises an exception when a function that requires an argument is called without any — even if the argument isn’t actually used...

Actually, you may have guessed the answer: the special thing about methods is that the instance object is passed as the first argument of the function. In our example, the call `x.f()` is exactly equivalent to `MyClass.f(x)`. In general, calling a method with a list of _n_ arguments is equivalent to calling the corresponding function with an argument list that is created by inserting the method’s instance object before the first argument.

Methods may call other methods by using method attributes of the self argument:

In [14]:
class Bag:
    def __init__(self):
        self.data = []

    def add(self, x):
        self.data.append(x)

    def addtwice(self, x):
        self.add(x)
        self.add(x)            # no return statement here!
ob = Bag()
ob.addtwice(7)                 # notice nothing is returned, even though the object is modified
ob.data

[7, 7]

## More examples

In [17]:
# Example 1
class MyStuff(object):
    def __init__(self):
        self.tangerine = 'And now a thousand years between'
    def apple(self):
        print('I am classy apples')

thing = MyStuff()

In [18]:
thing.apple()

I am classy apples


In [19]:
thing.tangerine

'And now a thousand years between'

In [20]:
# Example 2
class Animal(object):
    """Makes animals."""
    def __init__(self, name, age, hungry):
        self.name = name
        self.age = age
        self.is_hungry = hungry

zebra = Animal("Jeffrey", 2, True)
giraffe = Animal("Bruce", 1, False)
panda = Animal("Chad", 7, True)

print(zebra.name, zebra.age, zebra.is_hungry)
print(giraffe.name, giraffe.age, giraffe.is_hungry)
print(panda.name, panda.age, panda.is_hungry)

Jeffrey 2 True
Bruce 1 False
Chad 7 True


In [1]:
# Example 3
class Song(object):
    def __init__(self, lyrics):
        self.lyrics = lyrics
    def sing_me_a_song(self):
        for line in self.lyrics:
            print(line)            
happy_bday = Song(['Happy birthday to you',
                    'I don\'t want to get sued',
                    'So I\'ll stop right there'])
bulls_on_parade = Song(['They rally around the family',
                        'With a pocket full of shells'])

In [2]:
happy_bday.sing_me_a_song()

Happy birthday to you
I don't want to get sued
So I'll stop right there


In [3]:
bulls_on_parade.sing_me_a_song()

They rally around the family
With a pocket full of shells


In [5]:
# Example 4
class ShoppingCart(object):
    """Creates shopping cart objects for website users."""
    
    items_in_cart = {}
    def __init__(self, customer_name):
        self.customer_name = customer_name
    
    def add_item(self, product, price):
        """Add product to the cart."""
        if not product in self.items_in_cart:
            self.items_in_cart[product] = price
            print(product + " added.")
        else:
            print(product + " is already in the cart.")

    def remove_item(self, product):
        """Remove product from the cart."""
        if product in self.items_in_cart:
            del self.items_in_cart[product]
            print(product + " removed.")
        else:
            print(product + " is not in the cart.")

my_cart = ShoppingCart(customer_name="Brad")
my_cart.add_item("Ukelele", 10)

Ukelele added.


## Using docstrings in classes

It is good practice to use docstrings for both the class itself and methods:

In [22]:
class Point:
    """Represents a point in two-dimensional geometric coordinates."""
    def __init__(self, x=0, y=0):
        """Yes, you can but do not need to use docstrings under __init__."""
        self.move(x, y)
    def move(self, x, y):
        """Move the point to a new location in 2D space."""
        self.x = x
        self.y = y
    def reset(self):
        """'Reset the point back to the geometric origin."""
        self.move(0, 0)
        
help(Point)

Help on class Point in module __main__:

class Point(builtins.object)
 |  Represents a point in two-dimensional geometric coordinates.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x=0, y=0)
 |      Yes, you can but do not need to use docstrings under __init__.
 |  
 |  move(self, x, y)
 |      Move the point to a new location in 2D space.
 |  
 |  reset(self)
 |      'Reset the point back to the geometric origin.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



# Share data

Be careful about using a class variable without `self`.  For example, the tricks list in the following code should not be used as a class variable because just a single list would be shared by all Dog instances:

In [9]:
class Dog:

    tricks = []             # mistaken use of a class variable

    def __init__(self, name):
        self.name = name

    def add_trick(self, trick):
        self.tricks.append(trick)

d = Dog('Fido')
e = Dog('Buddy')
d.add_trick('roll over')
e.add_trick('play dead')
d.tricks                # unexpectedly shared by all dogs

['roll over', 'play dead']

Correct design of the class should use an instance variable instead:

In [10]:
class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick):
        self.tricks.append(trick)

d = Dog('Fido')
e = Dog('Buddy')
d.add_trick('roll over')
e.add_trick('play dead')
d.tricks
['roll over']
e.tricks

['play dead']

# `__repr__` and `__str__`

See also: [dunder methods](https://dbader.org/blog/python-dunder-methods)

When you define a custom class in Python and then try to print one of its instances to the console (or inspect it in an interpreter session), you get a relatively unsatisfying result:

In [1]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
my_car = Car('red', 37281)
print(my_car)
my_car

By default all you get is a string containing the class name and the id of the object instance (which is the object’s memory address in CPython.) That’s better than nothing, but it’s also not very useful.

`__str__` and `__repr__` are the Pythonic way to control how objects are converted to strings in different situations.

In [5]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

    def __str__(self):
        return f'a {self.color} car'

print(Car(color='red', mileage=5000))
Car(color='red', mileage=5000)

a red car


Inspecting the `Car` object in the console still gives us the previous result containing the object’s id. But printing the object resulted in the string returned by the  `__str__` method we added.

`__str__` is one of Python’s “dunder” (double-underscore) methods and gets called when you try to convert an object into a string through the various means that are available.

In [7]:
my_car = Car('red', 37281)
print(my_car)

a red car


In [8]:
str(my_car)

'a red car'

In [9]:
'{}'.format(my_car)

'a red car'

## Difference between `__repr__` and `__str__`

Notice in the above that _inspecting_ (not printing) `my_car` still gave the odd `<Car object at ...>` result.  That is because inspecting an object in a Python interpreter session simply prints the result of the object’s `__repr__`, which was not defined above.

In [10]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    def __repr__(self):
        return '__repr__ for Car'
    def __str__(self):
        return '__str__ for Car'
my_car = Car('red', 37281)
print(my_car)
my_car

__str__ for Car


__repr__ for Car

Note that the built in methods then call the dunder methods:

In [12]:
str(my_car)

'__str__ for Car'

In [13]:
repr(my_car)

'__repr__ for Car'

# Intermediate class conventions

You can build a class that accepts another class object as an input (see `.x` and `.y` below):

In [18]:
import math
class Point:
    def move(self, x, y):
        self.x = x
        self.y = y
    def reset(self):
        self.move(0, 0)
    def calculate_distance(self, other_point):
        return math.sqrt(
            (self.x - other_point.x)**2 +
            (self.y - other_point.y)**2)         # Pythagorean theorem
                                                 # Notice the references to other_point which is
                                                 # (implicitly) a Point class

In [9]:
point1 = Point()
point2 = Point()

point1.reset()
point2.move(5,0)

print(point2.calculate_distance(point1))

5.0


In [13]:
assert (point2.calculate_distance(point1) == point1.calculate_distance(point2))

point1.move(3,4)
print(point1.calculate_distance(point2))

4.47213595499958


# Class inheritance

The syntax for a derived class definition looks like this:

In [22]:
class BaseClassName:
    pass

class DerivedClassName(BaseClassName):
    #<statement-1>
    #.
    #.
    #.
    #<statement-N>
    pass

# If the base class is defined in another module:
# class DerivedClassName(modname.BaseClassName)

Recall a class defined in an above example:

In [15]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def set_age(self, age):
        self.age = age
    def set_major(self, major):
        self.major = major

Create a new class `MasterStudent` with the same methods and attributes as the previous one, but with an additional `internship` attribute:

In [8]:
class MasterStudent(Student):
    internship = 'mandatory, from March to June'

In [9]:
james = MasterStudent('james')

In [10]:
james.internship

'mandatory, from March to June'

In [15]:
james.set_age(23) # still need to define

In [16]:
james.age

23

Notes on inheritance:
* Inheritance creates a new instance of the base class.
* When the class object is constructed, the base class is remembered. This is used for resolving attribute references: if a requested attribute is not found in the class, the search proceeds to look in the base class.
* Derived classes may override methods of their base classes.
* The built-in [function](https://docs.python.org/3/library/functions.html#issubclass) `issubclass()` can check for class inheritance.

## From Shaw - _Learn Python the Hard Way_

There are three ways to make the parent and child class interact:
1. _Implied_ inheritance - leave the method alone
2. _Explicit override_ - redefine the method
3. `super`: alter the action on the parent

The below combines an example of all three:

In [5]:
class Parent(object):
    def implicit(self):
        print('Parent implicit()')
    def altered(self):
        print('Parent altered()')
    def override(self):
        print('Parent override()')


class Child(Parent):

    # 1. Implicit - don't write new method at all
    # 2. Override - define function with same name
    def override(self):
        print('Child override()')
    # 3. Alter *before* or *after* parent class's version runs
    #        using built-in `__super__()`
    def altered(self):
        print('Child, before parent altered()') # line 1
        super(Child, self).altered()            # line 2
        print('Child, after parent altered()')  # line 3

dad = Parent()
son = Child()

dad.implicit()
son.implicit()

dad.override()
son.override()

dad.altered()
son.altered()

Parent implicit()
Parent implicit()
Parent override()
Child override()
Parent altered()
Child, before parent altered()
Parent altered()
Child, after parent altered()


What happens in `Child.altered()` above?
1. Line 1: Override `Parent.altered()`.  `Child.altered()` runs.
2. Line 2: We use super to "get" the `Parent.altered()` method.  This is a lot like `getattr` that's aware of inheritance.  You can read it as, "call `super` with arguments `Child` and `self`, then call the function `altered` on whatever it returns.
3. At this point, `Parent.altered()` runs, and prints the "Parent altered()."  
4. Line 3: `Child.altered()` continues to print out after `super`.

The purpose of `super()` as it's used in this code block is to **alter the behavior _before or after_ the `Parent` class's version runs.**

`super()` can be seen in some cases as an **alternative to multiple inheritance.**  In this example,

```python
class SuperFun(Child, Uncle):
    pass
```

and you have implicit actions on any `SuperFun` instance, Python has to look up the possible function in the class hierarchy for both `Child` and `Uncle`, but needs to do this in a consistent order.  To do this, Python uses something called "method resolution order" (MRO) and an algorithm called C3 to determine order.  Alternatively, there is `super()`, which serves as a temporary in-place inheritance.

The most common use of `super()` is actually in `__init__` methods:  

In [10]:
class Child(Parent):
    def __init__(self, stuff):
        self.stuff = stuff
        super(Child, self).__init__()

This serves to:
- Set `stuff` as belong to `Child` (not parent)
- But then also have `Parent` initialize with `Parent.__init__()`

Note that in this example specifically, order does not particularly matter:

In [11]:
class Kid(Parent):
    def __init__(self, stuff):
        super(Child, self).__init__()
        self.stuff = stuff

## Composition

The alternative to inheritance is just to _use_ other classes and modules (call them directly), rather than rely on implicit inheritance.

In [8]:
class Other(object):
    def implicit(self):
        print('Other implicit()')
    def override(self):
        print('Other override()')
    def altered(self):
        print('Other altered()')

class Child(object):
    def __init__(self):
        self.other = Other()
    def implicit(self):
        return self.other.implicit()
    def override(self):
        print('Child override()')
    def altered(self):
        print('Child before other altered()')
        self.other.altered()
        print('Child after other altered()')

son = Child()
son.implicit()
son.override()
son.altered()

Other implicit()
Child override()
Child before other altered()
Other altered()
Child after other altered()


## Extending built-ins

One interesting use of inheritance is adding functionality to built-in classes.

In [3]:
class LongNameDict(dict):
    """Inherits from dict.  Extended with a `longest_key` attribute"""
    @property
    def longest_key(self):
        longest = None
        for key in self:
            if not longest or len(key) > len(longest):
                longest = key
        return longest
longkeys = LongNameDict({'a' : 1, 'longest' : 2, 'long' : 3})
longkeys.longest_key

'longest'

Most built-in types can be similarly extended. Commonly extended built-ins are `object`, `list`, `set`, `dict`, `file`, and `str`. Numerical types such as `int` and `float` are also occasionally inherited from.

When we have a built-in container object that we want to add functionality to, we have two options
1. We can either create a new object, which holds that container as an attribute (**composition**)
 * Best alternative if all we want to do is use the container to store some objects using that container's features
2. We can subclass the built-in object and add or adapt methods on it to do what we want (**inheritance**)
 * Use inheritance if we want to change the way the container actually works

### When to use inheritance or composition

1. Avoid multiple inheritance at all cost.  If you're stuck with it, be prepared to know class hierarchy and spend time finding where everything is coming from.
2. Use composition to package up code into modules that are used in many different unrelated places and situations.
3. Use inheritance only when there are clearly defined related reusable pieces of code that fit under a single common concept or if you have to because of something you're using.

## `class` versus `object`

The difference between a class and object can be confusing.  In layman's terms, a class is a generalized construct whereas an object is a specific instance of that class.  In a real-life example, `Fish` is a class, `Salmon` is its subclass, and a salmoned named `Mary` is the class instance object.

The confusing thing is that `object` is also the most base class in Python:

In [9]:
help(object)

Help on class object in module builtins:

class object
 |  The most base type



That is, `object` is the most base class from which all other classes inherit.  A `class` inherits from the `class` named `object` to make a `class`, which then becomes an object when instantiated into a class instance.

# More on `super()`

## Examples

In [6]:
# dumbed-down version of `logging.info()`
LOG = []

class LoggingDict(dict):
    """Extend the __setitem__ method of a dict.

    d['k'] = v under the hood calls d.__setitem__(k, v).  Here it is
        extended so that logging.info() logs a message with the added
        key, value pair.
    """

    def __setitem__(self, key, value):
        global LOG
        LOG.append('Setting %s to %r' % (key, value))
        super().__setitem__(key, value)

d = LoggingDict(a=1, x=5)
d['b'] = 2
d['c'] = 3
print(d)
print()
print(LOG)

{'a': 1, 'x': 5, 'b': 2, 'c': 3}

['Setting b to 2', 'Setting c to 3']


Before `super()` was introduced, we would have hardwired the call above with `dict.__setitem__(self, key, value)`. However, `super()` is better because it is a **computed indirect reference.**  In other words, if you rename the base class, the `super()` reference will automatically follow.

In [7]:
class NumpyArr(object):
    def __init__(self, x):
        self.x = x
    def array(self):
        return np.array(self.x)

class PandasDF(NumpyArr):
    def array(self):
        # Call parent's `array` and wrap it in a DataFrame
        return pd.DataFrame(super(PandasDF, self).array())
    
x = [[1, 2], [3, 4]]
PandasDF(x).array()

,0,1
0,1,2
1,3,4


What would happen if we called the above (incorrectly) without `super()`?

In [10]:
class PandasDF(NumpyArr):
    def array(self):
        return pd.DataFrame(self.array())

PandasDF(x).array()

RecursionError: maximum recursion depth exceeded

## Uses

`super()` is in the business of **delegating method calls to some class in the instance’s ancestor tree.**

There are two typical use cases for super:
1. In a class hierarchy with single inheritance, `super()` can be used to refer to parent classes without naming them explicitly, thus making the code more maintainable. This use closely parallels the use of super in other programming languages.  `super()` returns a proxy object that delegates method calls to a **parent or sibling** class of type. **This is useful for accessing inherited methods that have been overridden in a class.** The search order is same as that used by `getattr()` except that the type itself is skipped.
2. The second use case is to support **cooperative multiple inheritance** in a dynamic execution environment. This use case is unique to Python and is not found in statically compiled languages or languages that only support single inheritance. This makes it possible to implement “**diamond diagrams**” where multiple base classes implement the same method. Good design dictates that this method have the same calling signature in every case (because the order of calls is determined at runtime, because that order adapts to changes in the class hierarchy, and because that order can include sibling classes that are unknown prior to runtime).



## Change in syntax - Python2 to Python3

Note that the syntax changed in Python 3.0: you can just say `super().__init__()` instead of `super(ChildB, self).__init__()`.

In [2]:
class Parent(object):
    def name(self):
        print('Parent name()')


class Child(Parent):
    def name(self):
        print('Child name()')

    def use_old(self):
        super(Child, self).name()

    def also_use_old(self):
        # Equiv. to `use_old`
        super().name()

c = Child()
c.use_old()
c.also_use_old()

Parent name()
Parent name()


## Reminder: `__init__` is implicitly inherited

In fact, all methods including dunder methods are.

In the example below (at least), we don't need `super()`, if we want to inherit the parent's `__init__()` "as-is."

In [14]:
class A(object):
    def __init__(self):
        self.x = True

class B(A):
    # We don't need
    # super().__init__()
    # in this case!
    pass

b = B()
print(b.x)

True


Here is a modification in which we would indeed need it.  First the incorrect version:

In [20]:
class C(A):
    def __init__(self):
        # We are overriding here!
        # (Just like any other non-dunder method override)
        self.y = False

c = C()
print(c.x, c.y)  # will raise

AttributeError: 'C' object has no attribute 'x'

Corrected version:

In [19]:
class C(A):
    def __init__(self):
        super().__init__()
        self.y = False

c = C()
print(c.x, c.y)

True False


## Cooperative multiple inheritance

In [1]:
class Shape:
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        # This doesn't automatially call `object.__init__()`!
        # In this case, it actually calls `PositionMixin.__init__()`
        super().__init__(**kwds)


class PositionMixin:
    def __init__(self, x=0, y=0, **kwds):
        # This calls `object.__init__(**kwds)`
        super().__init__(**kwds)
        self.x = x
        self.y = y


class ColoredShape(Shape, PositionMixin):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)


myshape = ColoredShape('red', shapename='circle', x=1, y=1)


for attr in ['shapename', 'color', 'x', 'y']:
    print(getattr(myshape, attr))

circle
red
1
1


# Private variables

Most object-oriented programming languages have a concept of access control: some attributes and methods are marked private or protected.  Python doesn't do this.   Instead, it provides unenforced guidelines and best practices. 

Technically, all methods and attributes on a class are publicly available.  **“Private” instance variables that cannot be accessed except from inside an object don’t exist in Python.**

However, there are two conventions that are followed by most Python code:

1. **Single underscore (e.g. `_spam`)**:
    Python programmers will interpret this as "this is an internal variable, think three times before accessing it directly".
    
2. **Double underscore (`__spam`)**:
    This will perform **name mangling** on the object.  This basically means that the method can still be called by outside objects if they really want to do it, but it requires extra work and is a strong indicator that you demand that your attribute remains private.  An example is below.

In [24]:
class SecretString:
    """A not-at-all secure way to store a secret string."""
    def __init__(self, plain_string, pass_phrase):
        self.__plain_string = plain_string
        self.__pass_phrase = pass_phrase
    def decrypt(self, pass_phrase):
        """Only show the string if the pass_phrase is correct."""
        if pass_phrase == self.__pass_phrase:
            return self.__plain_string
        else:
            return None
        
secstr = SecretString(plain_string='acme', pass_phrase='antwerp') # tab completion will not show __xx attributes
secstr.__plain_string                                             # AttributeError coming

AttributeError: 'SecretString' object has no attribute '__plain_string'

In [25]:
secstr.decrypt('antwerp')

'acme'

However, this apparent security is easily penetrable.  This is Python name mangling at work. When we use a double underscore, the property is prefixed with `_<classname>`.

In [27]:
secstr._SecretString__plain_string

'acme'

## Properties

See also: 
* SO: [How does the @property decorator work?](https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work)
* programiz: [Python @property](https://www.programiz.com/python-programming/property)
* [documentation](https://docs.python.org/3/library/functions.html#property)

As discussed above, using an underscore or double underscore in Python does not technically make the variables private; it just _suggests_ that they be _treated as_ private.  A related concept is the `property` decorator.

Many object-oriented languages (Java is the most notorious) teach us to never access attributes directly. They insist that we write attribute access like this:

In [1]:
# separate get and set methods
class Color:
    def __init__(self, rgb_value, name):
        self._rgb_value = rgb_value
        self._name = name
    def set_name(self, name):
        self._name = name
    def get_name(self):
        return self._name

Basic Python favors a "direct access" version:

In [2]:
class Color:
    def __init__(self, rgb_value, name):
        self.rgb_value = rgb_value
        self.name = name
c = Color("#ff0000", "bright red")
c.name = "red" # set attr directly

**So why would anyone insist upon the method-based syntax?** Their reasoning is that someday we may want to add extra code when a value is set or retrieved.  Say that we went to change the `set_name()` method from above.

Now, in Java and similar languages, if we had written our original code to do direct attribute access, and then later changed it to a method like the preceding one, we'd have a problem: anyone who had written code that accessed the attribute directly would now have to access the method. If they don't change the access style from attribute access to a function call, their code will be broken. The mantra in these languages is that we should never make public members private. This doesn't make much sense in Python since there isn't any real concept of private members! 

**Python gives us the `property` keyword to make methods look like attributes.** We can therefore write our code to use direct member access, and if we unexpectedly need to alter the implementation to do some calculation when getting or setting that attribute's value, we can do so without changing the interface. Let's see how it looks:

In [3]:
class Color:
    def __init__(self, rgb_value, name):
        self.rgb_value = rgb_value
        self._name = name
    def _set_name(self, name):
        if not name:
            raise Exception("Invalid Name")
        self._name = name
    def _get_name(self):
        return self._name
    
    name = property(_get_name, _set_name)

**If we had started with the earlier non-method-based class, which set the name attribute directly, we could later change the code to look like the preceding one.** We first change the `name` attribute into a (semi-) private `_name` attribute. Then we add two more (semi-) private methods to get and set that variable, doing our validation when we set it.

So what does `property` do in the above?
* It creates a new attribute on the `Color` class called `name`, which now replaces the previous `name` attribute. 
* It sets this attribute to be a property, which calls the two methods we just created whenever the property is accessed or changed.

In [5]:
c  = Color("#0000ff", "bright red")
print(c.name)

bright red


In [7]:
c.name = "red"
print(c.name)

red


The [syntax](https://docs.python.org/3/library/functions.html#property) of property is

`class property(fget=None, fset=None, fdel=None, doc=None)`

where:
* `fget` is a function for getting an attribute value
* `fset` is a function for setting an attribute value
* `fdel` is a function for deleting an attribute value
* `doc` creates a docstring for the attribute.

So, as done in the above `Color` class, a typical use is to define a _managed attribute_:



In [8]:
class C:
    def __init__(self):
        self._x = None
    def getx(self):
        return self._x
    def setx(self, value):
        self._x = value
    def delx(self):
        del self._x
    x = property(getx, setx, delx, "I'm the 'x' property.")

Then, if `c` is an instance of `C`, **`c.x` will invoke the getter, `c.x = value` will invoke the setter and `del c.x` the deleter.**

If given, `doc` will be the docstring of the property attribute. **Otherwise, the property will copy `fget`’s docstring** (if it exists). This makes it possible to create read-only properties easily using `property()` as a decorator:





In [9]:
class Parrot:
    def __init__(self):
        self._voltage = 100000
    @property
    def voltage(self):
        """Get the current voltage."""
        return self._voltage

The `@property` decorator turns the `voltage()` method into a “getter” for a read-only attribute with the same name, and it sets the docstring for `voltage` to “Get the current voltage.”

To tie everything together: A property object has `getter`, `setter`, and `deleter` **methods** usable as decorators that create a copy of the property with the corresponding accessor function set to the decorated function.  This code is exactly equivalent to the first example of `C` above.

In [ ]:
class C:
    def __init__(self):
        self._x = None
    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x
    @x.setter
    def x(self, value):
        self._x = value
    @x.deleter
    def x(self):
        del self._x

## Deciding when to use properties

Remember that object are collections of data and associated behaviors.  But, the `property` built-in can be seen as clouding the division between behavior and data.

The fact that methods are just callable attributes, and properties are just customizable attributes can help us make this decision. **Methods should typically represent actions; things that can be done to, or performed by, the object. When you call a method, even with only one argument, it should do something. Method names are generally verbs.**

An example: a common need for custom behavior is caching a value that is difficult to calculate or expensive to look up (requiring, for example, a network request or database query). The goal is to store the value locally to avoid repeated calls to the expensive calculation.

In [10]:
from urllib.request import urlopen

class WebPage:
    def __init__(self, url):
        self.url = url
        self._content = None

    @property
    def content(self):
    # The first time the value is retrieved, we perform the lookup or 
    # calculation.  Then we could locally cache the value as a private 
    # attribute on our object (or in dedicated caching software), and the
    # next time the value is requested, we return the stored data
        if not self._content:
            print("Retrieving New Page...")
            self._content = urlopen(self.url).read()
        return self._content

import time

webpage = WebPage('http://ccphillips.net/')
now = time.time()
content1 = webpage.content
time.time() - now
now = time.time()
content2 = webpage.content
time.time() - now
content2 == content1

Retrieving New Page...


True

# Why is duplicate code a bad thing?

Don't Repeat Yourself (DRY) - a principle followed most religiously in Python.

Why is duplicate code a bad thing?  There are several reasons, but they all boil down to readability and maintainability.
* Keeping two similar pieces of code up to date can be a nightmare. We have to remember to update both sections whenever we update one of them, and we have to remember how the multiple sections differ so we can modify our changes when we are editing each of them.
* As soon as someone has to read and understand the code and they come across duplicate blocks, they are faced with a dilemma.  Code that might have made sense suddenly has to be understood. How is one section different from the other? How are they the same? Under what conditions is one section called? When do we call the other?